In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

from glob import glob
from tqdm import tqdm
from collections import deque

import requests
import pandas as pd
import numpy as np
import os

In [2]:
gp_path = f"{os.getcwd().split('EDA')[0]}\\EDA"
csv_path = f"{os.getcwd().split('EDA')[0]}\\web_crawling"
print(gp_path, csv_path, sep='\n')

c:\Users\reot1\OneDrive\바탕 화면\ml2\\EDA
c:\Users\reot1\OneDrive\바탕 화면\ml2\\web_crawling


In [3]:
raw_path = glob(f'{csv_path}\\**\\raw_*.csv', recursive=True)[0]
tag_path = glob(f'{csv_path}\\**\\tag_*.csv', recursive=True)[0]
print(raw_path, tag_path, sep='\n')

c:\Users\reot1\OneDrive\바탕 화면\ml2\\web_crawling\merge_dataset\raw_merge.csv
c:\Users\reot1\OneDrive\바탕 화면\ml2\\web_crawling\merge_dataset\tag_merge.csv


In [4]:
raw = pd.read_csv(raw_path, sep='\t')
tag = pd.read_csv(tag_path)

In [5]:
def check_info(df, type_=None):
    column_list = []
    if type_ == 'object':
        for col in df.columns:
            if df[col].dtype == type_:
                print(f'col :{col}\ttype :{df[col].dtype}\tNa :{sum(df[col].isna())}')
                column_list.append(col)
        return column_list
    
    elif type_ == 'int':
        for col in df.columns:
            if df[col].dtype == 'int64' or df[col].dtype == 'int32':
                print(f'col :{col}\ttype :{df[col].dtype}\tNa :{sum(df[col].isna())}')
                column_list.append(col)
        return column_list
    
    elif type_ == 'float':
        for col in df.columns:
            if df[col].dtype == 'float64' or df[col].dtype == 'float34':
                print(f'col :{col}\ttype :{df[col].dtype}\tNa :{sum(df[col].isna())}')
                column_list.append(col)
        return column_list
    
    else:
        for col in df.columns:
            print(f'col :{col}\ttype :{df[col].dtype}\tNa :{sum(df[col].isna())}')
    

In [7]:
# 'genre', 'platforms_num' 컬럼이 Na면 삭제
r_df = raw.copy()

In [10]:
check_info(r_df)

col :appid	type :int64	Na :0
col :24_Hour_Peak	type :int64	Na :0
col :All_time_peak	type :int64	Na :0
col :positive	type :int64	Na :0
col :negative	type :int64	Na :0
col :average_forever	type :int64	Na :0
col :average_2weeks	type :int64	Na :0
col :median_forever	type :int64	Na :0
col :median_2weeks	type :int64	Na :0
col :price	type :float64	Na :0
col :initialprice	type :float64	Na :0
col :genre	type :object	Na :0
col :num_lang	type :int64	Na :0
col :required_age	type :float64	Na :0
col :is_free	type :int64	Na :0
col :controller_support	type :int64	Na :0
col :dlc	type :float64	Na :0
col :metacritic	type :float64	Na :45005
col :metacritic_tf	type :int64	Na :0
col :windows	type :float64	Na :0
col :mac	type :float64	Na :0
col :linux	type :float64	Na :0
col :platforms_num	type :float64	Na :0
col :recommendations	type :float64	Na :0
col :achievements	type :float64	Na :0
col :new_date	type :object	Na :0
col :days_after_release	type :int64	Na :0
col :Review	type :object	Na :0
